In [ ]:
from matplotlib import pyplot as plt
from astropy.io import fits
import numpy as np
from scipy import ndimage
import re
import glob
import datetime

from astropy.utils.data import get_file_contents
import astropy.units as u
from astropy.time import Time

import pandas as pd

%matplotlib inline
from IPython.utils import io

In [ ]:
# Load the PANOPTES module dir
import sys
sys.path.append('../')

# For development
%load_ext autoreload
%autoreload 2

from panoptes.utils.images import *

In [ ]:
cr2_files = glob.glob('/home/wtgee/Desktop/*.cr2')
cr2_files.sort()
cr2_files

In [ ]:
pixel_scale = ((206.265) * (18.5 * u.micron) / (85 * u.mm)).cgs * u.arcsecond

In [ ]:
def compare(f0, f1):
    pgm_00 = cr2_to_pgm(f0)
    pgm_01 = cr2_to_pgm(f1)

#    exif_00 = read_exif(pgm_00)
    exif_01 = read_exif(f1)    
    
    raw_data_00 = read_pgm(pgm_00)
    raw_data_01 = read_pgm(pgm_01)
   
    x0, y0 = measure_offset(raw_data_00,raw_data_01, box_width=200)
    
    #with io.capture_output() as captured:    
    #    !/var/panoptes/bin/measure_offset $f0 $f1 
        
    #x1 = float(get_file_contents('xcent.txt'))
    #y1 = float(get_file_contents('ycent.txt'))

#    return ( x0, y0, x1, y1 )
    fmt = '%a %b %d %H:%M:%S %Y'
    t = datetime.datetime.strptime(exif_01.get('Timestamp'), fmt)
    return (x0, y0, t)

In [ ]:
comparison = []

for i in np.arange(len(cr2_files)):
    if i < len(cr2_files) - 1:
        comparison.append(compare(cr2_files[i], cr2_files[i+1]))

In [ ]:
x = [x[0] for x in comparison]
y = [x[1] for x in comparison]
dates = [x[2] for x in comparison]

In [ ]:
dt_index = pd.DatetimeIndex(dates)
dt_index

In [ ]:
x_offset = pd.DataFrame(x, dt_index)
y_offset = pd.DataFrame(y, dt_index)

In [ ]:
x_offset.plot()

In [ ]:
y_offset.plot()